In [1]:
!wget https://github.com/e9t/nsmc/raw/master/ratings.txt

--2025-10-16 00:46:43--  https://github.com/e9t/nsmc/raw/master/ratings.txt
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt [following]
--2025-10-16 00:46:44--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19515078 (19M) [text/plain]
Saving to: ‘ratings.txt’

ratings.txt         100%[===================>]  18.61M  49.3MB/s    in 0.4s    

2025-10-16 00:46:44 (49.3 MB/s) - ‘ratings.txt’ saved [19515078/19515078]



In [2]:
import os
import pandas as pd
file_list = os.listdir()

for file in file_list:
  if file == 'ratings.txt':
    df = pd.read_table((os.getcwd() + '/' + file), encoding='utf-8')
    df = df.dropna(how='any')
    break
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [3]:
with open((os.getcwd() + '/naver_review.txt'), 'w', encoding='utf-8') as f:
  f.write('\n'.join(df['document']))

In [4]:
!head naver_review.txt

어릴때보고 지금다시봐도 재밌어요ㅋㅋ
디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.
폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.
와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지
안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.
사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화
완전 감동입니다 다시봐도 감동
개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음
굿
바보가 아니라 병 쉰 인듯


In [6]:
from tokenizers import BertWordPieceTokenizer

tokenizer = BertWordPieceTokenizer(
    lowercase=False,
    strip_accents=False,
)
tokenizer

Tokenizer(vocabulary_size=0, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=False, lowercase=False, wordpieces_prefix=##)

In [8]:
# 말뭉치 데이터
data_file = 'naver_review.txt'
vocab_size = 30000
min_frequency = 2
initial_alphabet = []
limit_alphabet = 6000
special_tokens = [ '[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]']
wordpleces_orefux = '##'
show_progress=True

tokenizer.train(
    files=data_file,
    vocab_size=vocab_size,
    initial_alphabet=initial_alphabet,
    limit_alphabet=limit_alphabet,
    min_frequency=min_frequency,
    show_progress=show_progress,
    special_tokens=special_tokens,
)

vocab = tokenizer.get_vocab()
print('vocab size : ', len(vocab))
print(sorted(vocab, key=lambda x: vocab[x])[:20])

vocab size :  30000
['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/']


In [14]:
text = 'helodfklaf"jk hi my name is hyeon ji'

# 문장을 의미를 가진 최소의 단위로 나눔 (토큰화)
encoded = tokenizer.encode(text)

In [15]:
print(encoded.tokens)
print(encoded.ids)

# 이런 식으로 나누어진다~
# 몇번째 id 에 인간의 언어가 담긴 사전을 만든 것이다.
# 수치적인 의미는 아님. 사전에 몇 번째 순서에 기록된 것이라는 위치만 담고 있음

print(tokenizer.decode(encoded.ids))

['h', '##el', '##od', '##f', '##k', '##l', '##a', '##f', '"', 'j', '##k', 'h', '##i', 'my', 'na', '##m', '##e', 'is', 'h', '##ye', '##on', 'j', '##i']
[76, 13074, 10744, 4021, 3942, 3545, 3544, 4021, 6, 78, 3942, 76, 3522, 15443, 15878, 3638, 3639, 11495, 76, 26850, 8322, 78, 3522]
helodfklaf " jk hi my name is hyeon ji


## 임베딩

In [20]:
import torch
from torch import nn

# tokenizer -> 단어를 넣으면 그것에 대한 id가 나왔음
# id: embedding_vector -> id를 넣으면 그것의 벡터가 나옴
# 토큰화 -> 아이디값 embedding 에 넣기 -> vetor 화 -> RNN에 던진다

embedding_vector = nn.Embedding(vocab_size, 768) # 2**8 + 2**9 / 경험적인 측면에서 사용
# 토큰을 id로 -> id를 텐서로
token_id = tokenizer.token_to_id('h')
print('token_id : ', token_id)


input_id = torch.tensor([token_id], dtype=torch.long)
# 임의의 벡터 추출
vector = embedding_vector(input_id)
# print(vector.shape) # torch.Size([1, 768])
print(vector)

print(input_id.shape) # scalar

token_id :  76
torch.Size([1, 768])
tensor([[-5.4790e-01,  3.6083e-01, -3.1719e-01,  1.7665e+00, -2.5216e+00,
         -5.0133e-01,  6.0285e-01,  8.9851e-01,  1.1004e-01,  4.6479e-01,
          1.2792e+00,  4.6266e-01,  1.2012e+00, -2.6496e+00,  1.0388e+00,
         -1.1200e+00,  1.5805e+00,  1.2665e+00,  6.8033e-01,  1.6655e+00,
          1.0210e+00,  1.0830e-01,  1.0437e+00, -7.5912e-02, -6.8709e-01,
          5.3333e-02, -8.6008e-02,  6.7485e-01, -1.0419e-01,  1.2454e-01,
         -1.8763e+00, -1.5484e+00,  6.2340e-01,  7.9247e-01, -5.2700e-01,
         -6.1009e-01, -3.9607e-01, -8.1512e-01,  2.0955e-01, -3.8962e-02,
         -9.4820e-01, -6.8896e-01, -8.7108e-02, -1.1366e-01,  8.6514e-01,
         -7.0498e-01, -1.5257e+00,  1.8943e+00,  6.9588e-02,  8.0044e-01,
         -3.2980e-01,  3.4189e-01,  1.5010e+00,  6.5560e-01, -1.0211e+00,
          1.7091e+00, -1.3224e+00,  1.2903e+00, -1.0427e+00,  1.8832e-01,
         -9.8350e-01,  7.0976e-01, -1.5244e+00, -1.2923e+00, -2.6800e+00,
  

In [22]:
word_embeddings = nn.Embedding(vocab_size, 768)
input_size = word_embeddings.weight.size()[1]
# 은닉층의 차원
hidden_size = 1024
num_layers = 1
bidirectional = False

# rnn 초기화
rnn = nn.RNN(
    input_size=input_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=bidirectional,
)

# hidden state 초기화
hidden_state_shape = (num_layers * (2 if bidirectional else 1), hidden_size)
h_0 = torch.zeros(hidden_state_shape)

In [23]:
text = '나는 자고 싶다'

# 토큰화
encoded = tokenizer.encode(text)

# id만 가져오기
input_ids = encoded.ids

# tensor로 변환
input_ids = torch.tensor(input_ids, dtype=torch.long)
input_ids


tensor([ 6227, 10926,  6351])

In [25]:
input_embeds = word_embeddings(input_ids)
print(input_embeds.shape)

# rnn에 입력
outputs = rnn(input_embeds, h_0)
hidden_states = outputs[0] # 모든 hidden state
h_n = outputs[1] # 마지막 hidden state

assert torch.equal(hidden_states[-1].unsqueeze(0), h_n)
print(hidden_states)
print(hidden_states.shape)

torch.Size([3, 768])
tensor([[-0.0729,  0.4816,  0.3312,  ...,  0.2178,  0.6043, -0.5543],
        [-0.6321,  0.7617, -0.8116,  ...,  0.8444,  0.1107, -0.0261],
        [-0.4807, -0.3135, -0.0495,  ..., -0.8409,  0.6188,  0.3527]],
       grad_fn=<SqueezeBackward1>)
torch.Size([3, 1024])
